In [1]:
import numpy as np

# Load data
manta_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_manta.npy"
xiris_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_xiris.npy"
y_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_y.npy"
feats_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_feats.npy"

# load numpy arrays and display shapes
manta = np.load(manta_path)
xiris = np.load(xiris_path)
y = np.load(y_path)
print("manta shape: ", manta.shape)
print("xiris shape: ", xiris.shape)
print("y shape: ", y.shape) # laser power and velocity

#feats = np.load(feats_path)
#print("feats shape: ", feats.shape)


manta shape:  (9587, 320, 320)
xiris shape:  (9587, 320, 320)
y shape:  (9587, 2)


In [2]:
y = y[:, 0] # only use laser power
print("y shape: ", y.shape)

# normalize y
y = y / np.max(y)

y shape:  (9587,)


In [25]:
""" # get indices of each unique value
indices = []
for i in range(len(y_unique)):
    indices.append(np.where(y == y_unique[i])[0])
print("indices: ", indices)
y """

unique values in y:  [0.18181818 0.45454545 0.72727273 1.        ]
indices:  [array([   0,    1,    2, ..., 2413, 2414, 2415]), array([2416, 2417, 2418, ..., 4835, 4836, 4837]), array([4838, 4839, 4840, ..., 7205, 7206, 7207]), array([7208, 7209, 7210, ..., 9584, 9585, 9586])]


array([0.18181818, 0.18181818, 0.18181818, ..., 1.        , 1.        ,
       1.        ])

In [3]:
# unique values in y
y_unique = np.unique(y)
print("unique values in y: ", y_unique)

# encode y as integers based on unique values
y_encoded = np.zeros(y.shape)
for i in range(len(y_unique)):
    y_encoded[y == y_unique[i]] = i
print("y encoded: ", y_encoded)

unique values in y:  [0.18181818 0.45454545 0.72727273 1.        ]
y encoded:  [0. 0. 0. ... 3. 3. 3.]


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model

def create_encoder():
    inputs = Input(shape=(320, 320, 1))
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.Flatten()(x)
    model = Model(inputs=inputs, outputs=x)
    return model

# add projection head
def create_projection_head():
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu'))
    return model

input_shape = (320, 320, 1)

def add_projection_head(encoder):
    inputs = Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(128, activation='relu')(features)
    model = Model(inputs=inputs, outputs=outputs)
    return model


encoder = create_encoder()
encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 320, 320, 1)]     0         
                                                                 
 model (Functional)          (None, 369664)            55744     
                                                                 
 dense (Dense)               (None, 128)               47317120  
                                                                 
Total params: 47372864 (180.71 MB)
Trainable params: 47372864 (180.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-12-28 19:15:41.378299: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 19:15:41.439969: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 19:15:41.440126: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
# define loss function

In [ ]:
# complile model
encoder_with_projection_head.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# train model

In [ ]:


# Create two instances of the base network
base_network = create_base_network()
image1_network = models.Model(inputs=base_network.input, outputs=base_network.output)
image2_network = models.Model(inputs=base_network.input, outputs=base_network.output)

# Define the contrastive loss function
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, velocity1, laser_power2, velocity2):
        # Calculate the margins as functions of laser power and velocity
        margin_laser_power = tf.abs(laser_power1 - laser_power2)
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * (margin_square_laser_power + margin_square_velocity))
    
# Create input layers for laser power and velocity
laser_power_input1 = layers.Input(shape=(1,))
velocity_input1 = layers.Input(shape=(1,))
laser_power_input2 = layers.Input(shape=(1,))
velocity_input2 = layers.Input(shape=(1,))

# Connect input layers to base networks
image1_output = image1_network(base_network.output)
image2_output = image2_network(base_network.output)

# Concatenate image and feature outputs
merged_output = layers.concatenate([image1_output, image2_output, laser_power_input1, velocity_input1, laser_power_input2, velocity_input2])


# Compile the model with the contrastive loss function
model = models.Model(inputs=[image1_network.input, image2_network.input, laser_power1, velocity1, laser_power2, velocity2], outputs=distance)
model.compile(optimizer='adam', loss=ContrastiveLoss())


def calculate_similarity_indicator(laser_power1, velocity1, laser_power2, velocity2):
    # Calculate the difference in laser power and velocity
    difference_laser_power = abs(laser_power1 - laser_power2)
    difference_velocity = abs(velocity1 - velocity2)

    # Define a threshold for considering laser power and velocity as 'the same'
    threshold = 0.01

    # If the differences are below the threshold, return 1, else return 0
    if difference_laser_power < threshold and difference_velocity < threshold:
        return 1
    else:
        return 0

# Assuming you have a binary indicator representing whether the laser power and velocity are the same for each pair of images
similarity_indicator = calculate_similarity_indicator(laser_power1, velocity1, laser_power2, velocity2)

# Train the model
history = model.fit([image1, image2, laser_power1, velocity1, laser_power2, velocity2], similarity_indicator, epochs=10)


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models

2023-12-28 18:02:17.636605: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-28 18:02:17.720150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 18:02:17.720203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 18:02:17.723229: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 18:02:17.738875: I tensorflow/core/platform/cpu_feature_guar

In [12]:


# Assuming you have your images and features preprocessed and stored in numpy arrays
# image1, image2, laser_power, velocity

# Define the base network architecture
def create_base_network():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32))
    return model

# Create two instances of the base network
base_network = create_base_network()
image1_network = models.Model(inputs=base_network.input, outputs=base_network.output)
image2_network = models.Model(inputs=base_network.input, outputs=base_network.output)

# Define the distance layer
class DistanceLayer(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, image1, image2, laser_power1, velocity1, laser_power2, velocity2):
        image_distance = tf.reduce_sum(tf.square(image1 - image2), axis=-1)
        laser_power_distance = tf.square(laser_power1 - laser_power2)
        velocity_distance = tf.square(velocity1 - velocity2)
        return image_distance + laser_power_distance + velocity_distance
    

class ContrastiveLossLaserPower(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, laser_power2):
        # Calculate the margin as a function of laser power
        margin_laser_power = tf.abs(laser_power1 - laser_power2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_laser_power)

class ContrastiveLossVelocity(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, velocity1, velocity2):
        # Calculate the margin as a function of velocity
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_velocity)
    
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, velocity1, laser_power2, velocity2):
        # Calculate the margins as functions of laser power and velocity
        margin_laser_power = tf.abs(laser_power1 - laser_power2)
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * (margin_square_laser_power + margin_square_velocity))




# Create the distance layer
distance = distance_layer(image1_network.output, image2_network.output, laser_power1, velocity1, laser_power2, velocity2)
model = models.Model(inputs=[image1_network.input, image2_network.input, laser_power1, velocity1, laser_power2, velocity2], outputs=distance)

# Compile the model
model.compile(optimizer='adam', loss='contrastive_loss')  # You need to define 'contrastive_loss'

# Train the model
# model.fit([image1, image2, laser_power, velocity], labels, epochs=10)


2023-12-28 18:02:39.564482: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 18:02:39.667648: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 18:02:39.667897: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

NameError: name 'distance_layer' is not defined

In [ ]:
class CombinedLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.5):
        super().__init__()
        self.alpha = alpha
        self.contrastive_loss_laser_power = ContrastiveLossLaserPower()
        self.contrastive_loss_velocity = ContrastiveLossVelocity()

    def call(self, y_true, y_pred, laser_power1, laser_power2, velocity1, velocity2):
        loss_laser_power = self.contrastive_loss_laser_power(y_true, y_pred, laser_power1, laser_power2)
        loss_velocity = self.contrastive_loss_velocity(y_true, y_pred, velocity1, velocity2)
        return self.alpha * loss_laser_power + (1 - self.alpha) * loss_velocity


In [13]:
class ContrastiveLossLaserPower(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, laser_power2):
        # Calculate the margin as a function of laser power
        margin_laser_power = tf.abs(laser_power1 - laser_power2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_laser_power)

class ContrastiveLossVelocity(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, velocity1, velocity2):
        # Calculate the margin as a function of velocity
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_velocity)
    
model.compile(optimizer='adam', loss=[ContrastiveLossLaserPower(), ContrastiveLossVelocity()])

NameError: name 'model' is not defined